In [1]:
import sqlite3
import os
import pandas as pd
import json
import numpy as np
from datetime import datetime
from dateutil.parser import parse
from plotly import graph_objects as go
from plotly import io as pio
import xmltodict

In [51]:
crapome = pd.read_csv(os.path.join('data','crapome table.tsv'),sep='\t')
controls = pd.read_csv(os.path.join('data','control table.tsv'),sep='\t')
jsons = {}
for f in os.listdir('data'):
    if f.split('.')[-1]=='json':
        with open(os.path.join('data',f)) as fil:
            jsons[f'data_{f}'] = json.load(fil)

In [52]:
sets = {
    'VL GFP MAC3 AP': [
             'VL GFP MAC3-N AP-MS',
    ],
    'VL GFP MAC3 BioID': [
             'VL GFP MAC3-N BioID'
    ],
    'VL GFP MAC2 AP': [
             'VL GFP MAC2-C AP-MS',
             'VL GFP MAC2-N AP-MS',
    ],
    'VL GFP MAC2 BioID': [
             'VL GFP MAC2-C BioID',
             'VL GFP MAC2-N BioID'
    ],
    'VL GFP MAC AP': [
            'VL GFP MAC-C AP-MS',
            'VL GFP MAC-N AP-MS',
    ],
    'VL GFP MAC AP NLS': [
            'VL GFP MAC-MED-NLS AP-MS',
            'VL GFP MAC-MYC-NLS AP-MS',
            'VL GFP MAC-NLS AP-MS',
    ],
    'VL GFP MAC BioID': [
            'VL GFP MAC-C BioID',
            'VL GFP MAC-N BioID'
    ],
    'VL GFP MAC BioID NLS': [
            'VL GFP MAC-MED-NLS BioID',
            'VL GFP MAC-MYC-NLS BioID',
            'VL GFP MAC-NLS BioID'
    ],
    'Nesvilab': ['nesvilab']
}

In [53]:
crapome_tables = {}
columns = [
    'protein_id',
    'identified_in',
    'frequency',
    'spc_sum',
    'spc_avg',
    'spc_min',
    'spc_max',
    'spc_stdev']
types = [
    'TEXT PRIMARY KEY',
    'INTEGER NOT NULL',
    'REAL NOT NULL',
    'INTEGER NOT NULL',
    'REAL NOT NULL',
    'INTEGER NOT NULL',
    'INTEGER NOT NULL',
    'REAL NOT NULL'
]
crapome_entries = []
for setname, setcols in sets.items():
    all_cols = ['PROTID']
    defa = 1
    if 'MAC2' in setname: defa = 0 # default enabled value
    tablename = f'crapome_{setname}'.lower().replace(' ','_')
    for sc in setcols:
        all_cols.extend(jsons['data_crapome sets.json'][sc])
    all_cols = sorted(list(set(all_cols)))
    set_df = crapome[all_cols]
    set_df.index = set_df['PROTID']
    set_df = set_df.drop(columns=['PROTID']).replace(0,np.nan).dropna(how='all',axis=0).dropna(how='all',axis=1)
    nruns = set_df.shape[1]
    set_data = []
    for protid, row in set_df.iterrows():
        stdval = row.std()
        if pd.isna(stdval):
            stdval = -1
        set_data.append([protid, row.notna().sum(), row.notna().sum()/nruns,row.sum(), row.mean(), row.min(), row.max(), stdval])
    crapome_tables[tablename] = pd.DataFrame(columns=columns, data=set_data)
    crapome_entries.append([tablename, setname, nruns, 0, defa, tablename])
control_tables = {}
control_entries = []
for setname, setcols in sets.items():
    if setname == 'Nesvilab': continue
    all_cols = ['PROTID']
    defa = 1
    if 'MAC2' in setname: defa = 0
    tablename = f'control_{setname}'.lower().replace(' ','_')
    for sc in setcols:
        all_cols.extend(jsons['data_control sets.json'][sc])
    all_cols = sorted(list(set(all_cols)))
    set_df = controls[all_cols]
    set_df.index = set_df['PROTID']
    set_df = set_df.drop(columns=['PROTID']).replace(0,np.nan).dropna(how='all',axis=0).dropna(how='all',axis=1)
    nruns = set_df.shape[1]
    set_data = []
    for protid, row in set_df.iterrows():
        stdval = row.std()
        if pd.isna(stdval):
            stdval = -1
        set_data.append([protid, row.notna().sum(), row.notna().sum()/nruns,row.sum(), row.mean(), row.min(), row.max(), stdval])
    control_tables[tablename] = pd.DataFrame(columns=columns, data=set_data)
    control_entries.append([tablename, setname, nruns, 0, defa, tablename])


In [54]:
control_cols = ['control_set','control_set_name','runs','is_disabled','is_default','control_table_name']
crapome_cols = ['crapome_set','crapome_set_name','runs','is_disabled','is_default','crapome_table_name']
exts = ['TEXT PRIMARY KEY','TEXT NOT NULL','INTEGER NOT NULL','INTEGER NOT NULL','INTEGER NOT NULL','TEXT NOT NULL']

control_table_str =  [
        f'CREATE TABLE control_sets (',
    ]
for i, c in enumerate(control_cols):
    control_table_str.append(f'    {c} {exts[i]},',)
control_table_str = '\n'.join(control_table_str).strip(',')
control_table_str += '\n);'

crapome_table_str =  [
        f'CREATE TABLE crapome_sets (',
    ]
for i, c in enumerate(crapome_cols):
    crapome_table_str.append(f'    {c} {exts[i]},',)
crapome_table_str = '\n'.join(crapome_table_str).strip(',')
crapome_table_str += '\n);'

prot_cols = [
    'uniprot_id',
    'is_reviewed',
    'gene_name',
    'entry_name',
    'all_gene_names',
    'organism',
    'length',
    'sequence',
    'is_latest',
    'entry_source',
    'update_time'
]
prot_exts = [
    'TEXT PRIMARY KEY',
    'INTEGER NOT NULL',
    'TEXT NOT NULL',
    'TEXT NOT NULL',
    'TEXT NOT NULL',
    'TEXT NOT NULL',
    'INTEGER NOT NULL',
    'TEXT NOT NULL',
    'INTEGER NOT NULL',
    'TEXT NOT NULL',
    'TEXT NOT NULL'
]

prot_table_str =  [
        f'CREATE TABLE proteins (',
    ]
for i, c in enumerate(prot_cols):
    prot_table_str.append(f'    {c} {prot_exts[i]},',)
prot_table_str = '\n'.join(prot_table_str).strip(',')
prot_table_str += '\n);'

table_create_sql = [control_table_str, crapome_table_str, prot_table_str]

insert_sql = []

for vals in control_entries:
    tablename = vals[0]
    create_str = [
        f'CREATE TABLE {tablename} (',
    ]
    for i, c in enumerate(columns):
        create_str.append(f'    {c} {types[i]},',)
    create_str = '\n'.join(create_str).strip(',')
    create_str += '\n);'
    table_create_sql.append(create_str)
    add_str = [f'INSERT INTO control_sets ({", ".join(control_cols)}) VALUES ({", ".join(["?" for _ in control_cols])})', vals]
    insert_sql.append(add_str)
    for _, row in control_tables[tablename].iterrows():
        add_str = [f'INSERT INTO {tablename} ({", ".join(columns)}) VALUES ({", ".join(["?" for _ in columns])})', tuple(row.values)]
        insert_sql.append(add_str)
print('control:', len(insert_sql))


control: 37079


In [55]:

for vals in crapome_entries:
    tablename = vals[0]
    create_str = [
        f'CREATE TABLE {tablename} (',
    ]
    for i, c in enumerate(columns):
        create_str.append(f'    {c} {types[i]},',)
    create_str = '\n'.join(create_str).strip(',')
    create_str += '\n);'
    table_create_sql.append(create_str)
    add_str = [f'INSERT INTO crapome_sets ({", ".join(crapome_cols)}) VALUES({", ".join(["?" for _ in crapome_cols])})', vals]
    insert_sql.append(add_str)
    for _, row in crapome_tables[tablename].iterrows():
        add_str = [f'INSERT INTO {tablename} ({", ".join(columns)}) VALUES ({", ".join(["?" for _ in columns])})', tuple(row.values)]
        insert_sql.append(add_str)
print('crapome:',len(insert_sql))

crapome: 104782


In [56]:


uniprots = set(pd.read_csv(os.path.join('data_assets','db build files','uniprotkb_AND_reviewed_true_2023_09_04.tsv'),sep='\t')['Entry'].values)
prot = pd.read_csv(os.path.join('data_assets','db build files','uniprotkb_taxonomy_id_7711_AND_reviewed_2023_09_04.tsv'),sep='\t',index_col = 'Entry')
prot = pd.read_csv(os.path.join('data_assets','db build files','uniprotkb_taxonomy_id_7711_AND_reviewed_2023_09_04.tsv'),sep='\t',index_col = 'Entry')
for protid, row in prot.iterrows():
    gn = row['Gene Names (primary)']
    if pd.isna(gn):
        gn = row['Entry Name']
    gns = row['Gene Names']
    if pd.isna(gns):
        gns = row['Entry Name']
    row = row.fillna('')
    data = [
        protid,
        int(row['Reviewed']=='reviewed'),
        gn,
        row['Entry Name'],
        gns,
        row['Organism'],
        row['Length'],
        row['Sequence'],
        1,
        'uniprot_initial_download',
        datetime.today().strftime('%Y-%m-%d')
    ]
    add_str = f'INSERT INTO proteins ({", ".join(prot_cols)}) VALUES ({", ".join(["?" for _ in prot_cols])})'
    insert_sql.append([add_str, data])
print('protein:', len(insert_sql))


protein: 191629


In [57]:

cont_cols = [
    'uniprot_id',
    'is_reviewed',
    'gene_name',
    'entry_name',
    'all_gene_names',
    'organism',
    'length',
    'sequence',
    'entry_source',
    'contamination_source',
    'update_time'
]
cont_exts = [
    'TEXT PRIMARY KEY',
    'INTEGER NOT NULL',
    'TEXT NOT NULL',
    'TEXT NOT NULL',
    'TEXT NOT NULL',
    'TEXT NOT NULL',
    'INTEGER NOT NULL',
    'TEXT NOT NULL',
    
    'TEXT NOT NULL',
    'TEXT NOT NULL',
    'TEXT NOT NULL'
]

conts = pd.read_csv(os.path.join('Data','contaminant_list.tsv'),sep='\t')
conts = conts[~conts['Uniprot ID'].isin(['P0C1U8','Q2FZL2'])]
dd = pd.read_csv(os.path.join('data_assets','db build files','idmapping_2023_09_11.tsv'),sep='\t')
for _,row in dd.iterrows():
    conts.loc[conts[conts['Uniprot ID']==row['Entry']].index,'Length'] = row['Length']
dd2 = pd.read_csv(os.path.join('data_assets','db build files','idmapping_2023_09_121.tsv'),sep='\t')
for _, row in dd2.iterrows():
    ctloc = conts[conts['Uniprot ID']==row['From']]
    conts.loc[ctloc.index, 'Sequence'] = row['Sequence']
    conts.loc[ctloc.index, 'Gene names'] = row['Gene Names']
    conts.loc[ctloc.index, 'Length'] = row['Length']
    conts.loc[ctloc.index, 'Status'] = row['Reviewed']

In [58]:

updf = pd.read_csv(os.path.join('data_assets','db build files','uniprotkb_AND_reviewed_true_2023_09_04.tsv'),sep='\t')
seqs = {row['Entry']: row['Sequence'] for _, row in updf.iterrows()}
seq_col = []
for _,row in conts.iterrows():
    if row['Uniprot ID'] not in seqs:
        seq_col.append('')
    else:
        seq_col.append(seqs[row['Uniprot ID']])
conts['Sequence'] = seq_col
conts['Length'] = conts['Length'].fillna(1).astype(int)
for i, row in conts[conts['Gene names'].isna()].iterrows():
    conts.loc[i, 'Gene names'] = f'{row["Protein names"]}({row["Uniprot ID"]})'
conts['Organism'] = conts['Organism'].fillna('None')
conts['Sequence'] = conts['Sequence'].fillna('Unknown')
conts['Sequence'] = conts['Sequence'].fillna('Unknown')
conts['Source of Contamination'] = conts['Source of Contamination'].fillna('Unspecified')
cont_table_str =  [
    f'CREATE TABLE contaminants (',
]


In [59]:
for i, c in enumerate(cont_cols):
    cont_table_str.append(f'    {c} {cont_exts[i]},',)
cont_table_str = '\n'.join(cont_table_str).strip(',')
cont_table_str += '\n);'
for _, row in conts.iterrows():
    gn = row['Gene names']
    if not 'Uncharac' in gn:
        gn = gn.split()[0]
    gns = row['Gene names']
    data = [
        row['Uniprot ID'],
        int(row['Status']=='reviewed'),
        gn,
        row['Entry name'],
        gns,
        row['Organism'],
        row['Length'],
        row['Sequence'],
        row['DataBase'],
        row['Source of Contamination'],
        datetime.today().strftime('%Y-%m-%d')
    ]
    add_str = f'INSERT INTO contaminants ({", ".join(cont_cols)}) VALUES ({", ".join(["?" for _ in cont_cols])})'
    insert_sql.append([add_str, data])
table_create_sql.append(cont_table_str)
print('contaminants:',len(insert_sql))

contaminants: 192076


In [60]:
tictable_create = ['CREATE TABLE IF NOT EXISTS ms_runs (']
tic_cols = [
    'run_id TEXT PRIMARY KEY',
    'sample_name TEXT NOT NULL',
    'run_name TEXT NOT NULL',
    'run_time TEXT NOT NULL',
    'instrument TEXT NOT NULL',
    'author TEXT NOT NULL',
    'sample_type TEXT NOT NULL',
    'run_type TEXT NOT NULL',
    'lc_method TEXT NOT NULL',
    'ms_method TEXT NOT NULL',
    'bait TEXT',
    'bait_uniprot TEXT',
    'chromatogram_max_time INTEGER NOT NULL',
    'cell_line TEXT',
    'project TEXT',
    'author_notes TEXT',
    'bait_tag TEXT',
]
for col in tic_cols:
    tictable_create.append(f'    {col},')
for col in  [
        'auc REAL NOT NULL',
        'intercepts INTEGER NOT NULL',
        'avg_peaks_per_timepoint REAL NOT NULL',
        'mean_intensity INTEGER NOT NULL',
        'max_intensity INTEGER NOT NULL',
        'json TEXT NOT NULL',
        'trace TEXT NOT NULL',
    ]:
    tic_cols.extend([
        f'bpc_{col}',
        f'msn_{col}',
        f'tic_{col}'
    ])
    tictable_create.extend([f'    {c},' for c in tic_cols[-3:]])
tictable_create = '\n'.join(tictable_create).strip(',')
tictable_create += '\n);'
table_create_sql.append(tictable_create)

## Run the dig_tic.py script at this point from an account with access to .d storage folder.

In [61]:
new_cols = []
for typ in ['BPC','TIC','MSn']:
    for key in ['auc','intercepts','peaks_per_timepoint','mean_intensity','max_intensity', 'json','trace', 'intercept_json']:
        new_cols.append(f'{typ}_{key}')
col_renames = {c: c.replace('peaks_per_timepoint','avg_peaks_per_timepoint') for c in new_cols}
tic_dir = os.path.join('data_assets','TIC data')
tic_data_files = [os.path.join(tic_dir, f) for f in os.listdir(tic_dir) if 'TIC data_' in f]
tic_data = pd.concat([pd.read_csv(f,sep='\t') for f in tic_data_files]).drop(columns='Unnamed: 0')
new_data = [[] for _ in new_cols]
for _,row in tic_data.iterrows():
    dic = json.loads(row['chromatograms'])
    if len(dic['json'].keys()) == 0:
        for i, nc in enumerate(new_cols):
            new_data[i].append(np.nan)
    else:
        for i, nc in enumerate(new_cols):
            typ, key = nc.split('_',maxsplit=1)
            new_data[i].append(dic[key][typ])
for i, nc in enumerate(new_cols):
    tic_data[nc] = new_data[i]
if not 'chromatogram_max_time' in tic_data.columns:
    tvals = []
    for _,row in tic_data.iterrows():
        tvals.append(pd.Series(row['BPC_json']).index.max())
    tic_data['chromatogram_max_time'] = tvals
dic_cols = [c for c in tic_data.columns if 'json' in c]
for d in dic_cols:
    nvals = [json.dumps(v) if type(v) is dict else str(v).replace('nan','') for v in tic_data[d].values]
    tic_data[d] = nvals
tic_data = tic_data.rename(columns=col_renames)
tic_data = tic_data.rename(columns={
    c: c.lower() for c in tic_data.columns
})
tic_data = tic_data.rename(columns={
    'sample_id': 'run_id', 'datafolder_name': 'run_name','HyStar_LC_Method_Name'.lower(): 'lc_method', 'HyStar_MS_Method_Name'.lower(): 'ms_method'
})
tic_data['instrument'] = 'timsTOF pro 2'
tic_data['run_id'] = tic_data['run_id'].astype(int)
tic_data.head(2)


,sample_name,run_id,run_name,lc_method,ms_method,run_time,chromatograms,chromatogram_max_time,bpc_auc,bpc_intercepts,...,tic_intercept_json,msn_auc,msn_intercepts,msn_avg_peaks_per_timepoint,msn_mean_intensity,msn_max_intensity,msn_json,msn_trace,msn_intercept_json,instrument
0,phospho2,822,phospho2_S1-A2_1_822.d,Evosep 60 samples/day,DDA PASEF-short_gradient_0.5sec_cycletime.m,2021-08-30_11:42:13,"{""json"": {""BPC"": {""0"": 6925.666666666667, ""5"":...",NaN,3.703605e+10,69.0,...,"{""255"": 11062370.770514324, ""990"": 11062370.77...",2.680163e+10,26.0,37.249012,178918.984666,526189.324324,"{""0"": 2815.0, ""5"": 2955.153846153846, ""10"": 32...","{""name"":""822"",""type"":""scatter"",""x"":[0,5,10,15,...","{""125"": 178918.98466587564, ""130"": 178918.9846...",timsTOF pro 2
1,phospho1,821,phospho1_S1-A1_1_821.d,Evosep 60 samples/day,DDA PASEF-short_gradient_0.5sec_cycletime.m,2021-08-30_11:18:35,"{""json"": {""BPC"": {""0"": 7314.0, ""5"": 7479.8, ""1...",NaN,2.921505e+10,67.0,...,"{""205"": 10790593.88314187, ""215"": 10790593.883...",1.958429e+10,34.0,36.909091,139397.133433,486652.473684,"{""0"": 2974.0, ""5"": 3302.3333333333335, ""10"": 2...","{""name"":""821"",""type"":""scatter"",""x"":[0,5,10,15,...","{""140"": 139397.13343284556, ""155"": 139397.1334...",timsTOF pro 2


In [62]:
runlist = pd.read_excel(os.path.join('..','..','..','combined runlist.xlsx'))

In [63]:
nvals = {c: [] for c in [r for r in tic_cols if r.split()[0] not in tic_data.columns]}
for _,row in tic_data.iterrows():
    run = runlist[runlist['Raw file']==row['run_name']]
    if run.shape[0]==0:
        for key in nvals.keys():
            nvals[key].append('')
    else:
        run = run.iloc[0]
        nvals['author TEXT NOT NULL'].append(run['Who'])
        nvals['sample_type TEXT NOT NULL'].append(run['Sample type'])
        nvals['run_type TEXT NOT NULL'].append(run['Run type'])
        nvals['cell_line TEXT'].append(run['Cell line / material'])
        nvals['project TEXT'].append(run['Project'])
        nvals['author_notes TEXT'].append(run['Notes'])
        nvals['bait_tag TEXT'].append(run['tag'])
        nvals['bait TEXT'].append(run['Bait / other uniprot or ID'])
        if run['Bait / other uniprot or ID'] in uniprots:
            nvals['bait_uniprot TEXT'].append(run['Bait / other uniprot or ID'])
        else:
            nvals['bait_uniprot TEXT'].append('')
for c, vals in nvals.items():
    tic_data[c.split()[0]] = vals
tic_data = tic_data[tic_data['chromatogram_max_time']!='']
tic_data = tic_data[tic_data['chromatogram_max_time']!=0]
tic_data = tic_data[tic_data['chromatogram_max_time'].notna()]
tic_data = tic_data.drop_duplicates()

In [64]:
print(len(insert_sql))
for _,row in tic_data.iterrows():
    data = [
        row[c.split()[0]] for c in tic_cols
    ]
    add_str = f'INSERT INTO ms_runs ({", ".join([c.split()[0] for c in tic_cols])}) VALUES ({", ".join(["?" for _ in tic_cols])})'
    insert_sql.append([add_str, data])
print(len(insert_sql))

192076
193926


aiemmin:
- control: 37079
- crapome: 104782
- protein: 191629
- contaminants: 192076

- 192076
- 193926



In [65]:
inttable_create = ['CREATE TABLE IF NOT EXISTS known_interactions (']
inttable_cols = [
    'interaction TEXT PRIMARY KEY',
    'uniprot_id_a TEXT NOT NULL',
    'uniprot_id_b TEXT NOT NULL',
    'uniprot_id_a_noiso TEXT NOT NULL',
    'uniprot_id_b_noiso TEXT NOT NULL',
    'source_database TEXT NOT NULL',
    'isoform_a TEXT',
    'isoform_b TEXT',
    'experimental_role_interactor_a TEXT',
    'interaction_detection_method TEXT',
    'publication_identifier TEXT',
    'biological_role_interactor_b TEXT',
    'annotation_interactor_a TEXT',
    'confidence_value TEXT',
    'interaction_type TEXT',
    'experimental_role_interactor_b TEXT',
    'annotation_interactor_b TEXT',
    'biological_role_interactor_a TEXT',
    'notes TEXT',
    'update_time TEXT',
]
for col in inttable_cols:
    inttable_create.append(f'    {col},')
inttable_create = '\n'.join(inttable_create).strip(',')
inttable_create += '\n);'
table_create_sql.append(inttable_create)
print(len(insert_sql))

for int_df_name in ['intact.tsv']:
    int_df = pd.read_csv(int_df_name,sep='\t')
    int_df_slim = int_df[int_df['uniprot_id_a'].isin(prot.index.values) & int_df['uniprot_id_b'].isin(prot.index.values)]
    for _,row in int_df_slim.iterrows():
        data = [
            row[c.split()[0]] for c in inttable_cols
        ]
        add_str = f'INSERT INTO known_interactions ({", ".join([c.split()[0] for c in inttable_cols])}) VALUES ({", ".join(["?" for _ in inttable_cols])})'
        insert_sql.append([add_str, data])
    print(int_df_name, len(insert_sql))

193926
intact.tsv 572328


In [75]:

# Connect to the database (create it if it doesn't exist)
conn = sqlite3.connect(os.path.join('data','proteogyver6.db'))
# Create a cursor object
cursor = conn.cursor()
start = datetime.now()
for create_table_str in table_create_sql:
    cursor.execute(create_table_str)
for insert_str, insert_data in is2:
    cursor.execute(insert_str, insert_data)
print('Table creation and data insertion took', (datetime.now() - start).seconds, 'seconds')
# Commit changes and close the connection
conn.commit()
conn.close()


Table creation and data insertion took 4 seconds


In [102]:
crapome.head()

,PROTID,AP_GFP_C_1_S2-A2_1_6530,AP_GFP_C_2_S2-A3_1_6531,AP_GFP_C_3_S2-A4_1_6532,AP_GFP_MED_NLS_1_S2-B2_1_6542,AP_GFP_MED_NLS_2_S2-B3_1_6543,AP_GFP_MED_NLS_3_S2-B4_1_6544,AP_GFP_MYC_NLS_1_S2-B6_1_6546,AP_GFP_MYC_NLS_2_S2-B7_1_6547,AP_GFP_MYC_NLS_3_S2-B8_1_6548,...,VL GFP MAC-N BioID AvgSpc,VL GFP MAC-N BioID Frequency,VL GFP MAC2-C BioID AvgSpc,VL GFP MAC2-C BioID Frequency,VL GFP MAC2-N BioID AvgSpc,VL GFP MAC2-N BioID Frequency,VL GFP MAC3-N BioID AvgSpc,VL GFP MAC3-N BioID Frequency,nesvilab AvgSpc,nesvilab Frequency
0,A0A024QZ33,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0,NaN,0,NaN,0,NaN,0,2.580153,18
1,A0A024QZ42,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0,NaN,0,NaN,0,NaN,0,2.981308,15
2,A0A024QZP7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0,NaN,0,NaN,0,NaN,0,4.148936,39
3,A0A024QZX5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0,NaN,0,NaN,0,NaN,0,3.150685,10
4,A0A024R0Y4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0,NaN,0,NaN,0,NaN,0,1.000000,0


In [103]:
random.randint(1,132)

32

In [109]:
data = []
for i in random.sample(list(d.index), 1000):
    data.append([d.loc[i]['uniprot_id_a'], d.loc[i]['uniprot_id_b'], random.randint(1,231)])
ios = set(d['uniprot_id_a'].values)
for i in crapome['PROTID'].values:
    if i not in ios:
        data.append([i, random.choice(d['uniprot_id_b'].values), random.randint(1,231)])
df = pd.DataFrame(data=data,columns=['Bait','Prey','AvgSpec'])

In [117]:
h = pd.merge(df,d,left_on=['Bait','Prey'],right_on=['uniprot_id_a','uniprot_id_b'],how='left')

In [119]:
df.shape

(29442, 3)

In [116]:
d.shape

(217400, 20)

In [118]:
h.shape

(29442, 23)

In [128]:
hc = h['Bait'].value_counts().sort_index()
hd = h[h['update_time'].isna()]['Bait'].value_counts().sort_index()

In [135]:
hc = pd.Series([1,2,1,2,1,2,1,2,1,2,3,4,4,3,3,3,4,4,4]).value_counts()
hd = pd.Series([1,2,1,2,1,2,1,2,1,2,3]).value_counts()

In [139]:
h['Known interaction'] = h['update_time'].notna()

,Bait,Known interaction,Prey count
0,P05412,True,10
1,P62993,True,7
2,P35226,True,6
3,Q9Y265,True,6
4,Q13618,True,5
...,...,...,...
29197,F2Z2R5,False,1
29198,F2Z2R1,False,1
29199,F2Z2Q9,False,1
29200,F2Z2Q6,False,1


In [138]:
h[['Bait','update_time']].fillna('').value_counts()

Bait    update_time
P05412  2023-09-20     10
P62993  2023-09-20      7
P35226  2023-09-20      6
Q9Y265  2023-09-20      6
Q13618  2023-09-20      5
                       ..
F2Z2R5                  1
F2Z2R1                  1
F2Z2Q9                  1
F2Z2Q6                  1
X6RM59                  1
Length: 29202, dtype: int64

In [136]:
hc-hd

1    0.0
2    0.0
3    3.0
4    NaN
dtype: float64

In [120]:
h

,Bait,Prey,AvgSpec,interaction,uniprot_id_a,uniprot_id_b,uniprot_id_a_noiso,uniprot_id_b_noiso,source_database,isoform_a,...,publication_identifier,biological_role_interactor_b,annotation_interactor_a,confidence_value,interaction_type,experimental_role_interactor_b,annotation_interactor_b,biological_role_interactor_a,notes,update_time
0,Q15046,P04406,170,Q15046_-_P04406,Q15046,P04406,Q15046,P04406,IntAct,None,...,pubmed:21900206|imex:IM-16799|mint:MINT-808964...,"psi-mi:""MI:0499""(unspecified role)",-,intact-miscore:0.37,"psi-mi:""MI:0915""(physical association)","psi-mi:""MI:0496""(bait)",-,"psi-mi:""MI:0499""(unspecified role)",None,2023-09-20
1,P27635,O95816,9,P27635_-_O95816,P27635,O95816,P27635,O95816,IntAct,None,...,pubmed:18457437|mint:MINT-6729753|imex:IM-22660,"psi-mi:""MI:0499""(unspecified role)",-,intact-miscore:0.35,"psi-mi:""MI:0914""(association)","psi-mi:""MI:0496""(bait)",-,"psi-mi:""MI:0499""(unspecified role)",None,2023-09-20
2,P04040,Q9UQR0,214,P04040_-_Q9UQR0,P04040,Q9UQR0,P04040,Q9UQR0,IntAct,None,...,pubmed:29568061|imex:IM-26301,"psi-mi:""MI:0499""(unspecified role)",-,intact-miscore:0.27,"psi-mi:""MI:2364""(proximity)","psi-mi:""MI:0498""(prey)",-,"psi-mi:""MI:0499""(unspecified role)",None,2023-09-20
3,Q9NPJ6,O75935,196,Q9NPJ6_-_O75935,Q9NPJ6,O75935,Q9NPJ6,O75935,IntAct,None,...,pubmed:25281560|imex:IM-27803,"psi-mi:""MI:0499""(unspecified role)",-,intact-miscore:0.27,"psi-mi:""MI:2364""(proximity)","psi-mi:""MI:0498""(prey)",-,"psi-mi:""MI:0499""(unspecified role)",None,2023-09-20
4,P61088,P22314,157,P61088_-_P22314,P61088,P22314,P61088,P22314,IntAct,None,...,pubmed:27173435|imex:IM-25054|pubmed:unassigne...,"psi-mi:""MI:0499""(unspecified role)|psi-mi:""MI:...",-|-,"intact-miscore:0.51|author score:""Socio-affini...","psi-mi:""MI:0914""(association)|psi-mi:""MI:0915""...","psi-mi:""MI:0498""(prey)|psi-mi:""MI:0497""(neutra...",-|-,"psi-mi:""MI:0499""(unspecified role)|psi-mi:""MI:...",None,2023-09-20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29437,X6RLT1,Q99689,46,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29438,X6RLV5,P11182,113,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29439,X6RLX0,P20674,149,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29440,X6RM00,Q9UJW9,98,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [110]:
df

,Bait,Prey,AvgSpec
0,Q15046,P04406,170
1,P27635,O95816,9
2,P04040,Q9UQR0,214
3,Q9NPJ6,O75935,196
4,P61088,P22314,157
...,...,...,...
29437,X6RLT1,Q99689,46
29438,X6RLV5,P11182,113
29439,X6RLX0,P20674,149
29440,X6RM00,Q9UJW9,98


In [101]:
d

,interaction,uniprot_id_a,uniprot_id_b,uniprot_id_a_noiso,uniprot_id_b_noiso,source_database,isoform_a,isoform_b,experimental_role_interactor_a,interaction_detection_method,publication_identifier,biological_role_interactor_b,annotation_interactor_a,confidence_value,interaction_type,experimental_role_interactor_b,annotation_interactor_b,biological_role_interactor_a,notes,update_time
0,Q99961_-_O43426,Q99961,O43426,Q99961,O43426,IntAct,None,None,"psi-mi:""MI:0498""(prey)|psi-mi:""MI:0498""(prey)|...","psi-mi:""MI:0081""(peptide array)|psi-mi:""MI:008...",pubmed:10542231|mint:MINT-5211933|pubmed:10542...,"psi-mi:""MI:0499""(unspecified role)|psi-mi:""MI:...",crc64:6E5BE978BC955077|crc64:6E5BE978BC955077|...,intact-miscore:0.44|intact-miscore:0.44|intact...,"psi-mi:""MI:0407""(direct interaction)|psi-mi:""M...","psi-mi:""MI:0496""(bait)|psi-mi:""MI:0496""(bait)|...",-|-|-,"psi-mi:""MI:0499""(unspecified role)|psi-mi:""MI:...",None,2023-09-20
1,Q99961_-_Q05193,Q99961,Q05193,Q99961,Q05193,IntAct,None,None,"psi-mi:""MI:0498""(prey)|psi-mi:""MI:0498""(prey)","psi-mi:""MI:0081""(peptide array)|psi-mi:""MI:008...",pubmed:10542231|mint:MINT-5211933|pubmed:10542...,"psi-mi:""MI:0499""(unspecified role)|psi-mi:""MI:...",crc64:6E5BE978BC955077|crc64:6E5BE978BC955077,intact-miscore:0.44|intact-miscore:0.44,"psi-mi:""MI:0407""(direct interaction)|psi-mi:""M...","psi-mi:""MI:0496""(bait)|psi-mi:""MI:0496""(bait)",-|-,"psi-mi:""MI:0499""(unspecified role)|psi-mi:""MI:...",None,2023-09-20
2,Q99961_-_P50570,Q99961,P50570,Q99961,P50570,IntAct,None,None,"psi-mi:""MI:0498""(prey)","psi-mi:""MI:0081""(peptide array)",pubmed:10542231|mint:MINT-5211933,"psi-mi:""MI:0499""(unspecified role)",crc64:6E5BE978BC955077,intact-miscore:0.35,"psi-mi:""MI:0914""(association)","psi-mi:""MI:0496""(bait)",-,"psi-mi:""MI:0499""(unspecified role)",None,2023-09-20
3,Q99961_-_Q9UQ16,Q99961,Q9UQ16,Q99961,Q9UQ16,IntAct,None,None,"psi-mi:""MI:0498""(prey)","psi-mi:""MI:0081""(peptide array)",pubmed:10542231|mint:MINT-5211933,"psi-mi:""MI:0499""(unspecified role)",crc64:6E5BE978BC955077,intact-miscore:0.35,"psi-mi:""MI:0914""(association)","psi-mi:""MI:0496""(bait)",-,"psi-mi:""MI:0499""(unspecified role)",None,2023-09-20
4,Q99961_-_Q96B97,Q99961,Q96B97,Q99961,Q96B97,IntAct,None,None,"psi-mi:""MI:0498""(prey)|psi-mi:""MI:0498""(prey)|...","psi-mi:""MI:0019""(coimmunoprecipitation)|psi-mi...",pubmed:11894095|mint:MINT-5215184|pubmed:11894...,"psi-mi:""MI:0499""(unspecified role)|psi-mi:""MI:...","crc64:6E5BE978BC955077|comment:""Stoichiometry:...",intact-miscore:0.59|intact-miscore:0.59|intact...,"psi-mi:""MI:0915""(physical association)|psi-mi:...","psi-mi:""MI:0496""(bait)|psi-mi:""MI:0496""(bait)|...","comment:""Stoichiometry: 1.0""|comment:""Stoichio...","psi-mi:""MI:0499""(unspecified role)|psi-mi:""MI:...",None,2023-09-20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
217395,Q9H7B2_-_P78362,Q9H7B2,P78362,Q9H7B2,P78362,IntAct,None,None,"psi-mi:""MI:0498""(prey)","psi-mi:""MI:0096""(pull down)",pubmed:32707033|imex:IM-29257|doi:10.1016/j.mo...,"psi-mi:""MI:0499""(unspecified role)",-,intact-miscore:0.35,"psi-mi:""MI:0914""(association)","psi-mi:""MI:0496""(bait)",-,"psi-mi:""MI:0499""(unspecified role)",None,2023-09-20
217396,Q9H7B2_-_Q8NCK7,Q9H7B2,Q8NCK7,Q9H7B2,Q8NCK7,IntAct,None,None,"psi-mi:""MI:0498""(prey)","psi-mi:""MI:0007""(anti tag coimmunoprecipitation)",imex:IM-29297|pubmed:28666119,"psi-mi:""MI:0499""(unspecified role)",-,intact-miscore:0.35,"psi-mi:""MI:0914""(association)","psi-mi:""MI:0496""(bait)",-,"psi-mi:""MI:0499""(unspecified role)",None,2023-09-20
217397,Q9H7B2_-_P30530,Q9H7B2,P30530,Q9H7B2,P30530,IntAct,None,None,"psi-mi:""MI:0498""(prey)","psi-mi:""MI:1314""(proximity-dependent biotin id...",pubmed:35384245|imex:IM-29494,"psi-mi:""MI:0499""(unspecified role)",-,intact-miscore:0.27,"psi-mi:""MI:2364""(proximity)","psi-mi:""MI:0496""(bait)",-,"psi-mi:""MI:0499""(unspecified role)",None,2023-09-20
217398,Q9H7B2_-